In [1]:
import gym
import tensorflow as tf
import numpy as np
import copy
import random
from collections import deque
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
#env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
env.render()


SFFF
FHFH
FFFH
HFFG


In [14]:
lr=0.005


In [3]:
inputdim=env.observation_space.n
outputdim=env.action_space.n
hiddendim=100
print(inputdim, outputdim)
X=tf.placeholder(tf.float32, shape = [None,inputdim],name="X")
Y=tf.placeholder(tf.float32, shape = [None,outputdim],name="Y")
# Y=tf.placeholder(tf.float32,shape=[None,1],name="Y")
# action=tf.placeholder(tf.int32,shape=[1],name="action")

weight1=tf.get_variable("w1", initializer=tf.keras.initializers.he_uniform(), shape=[inputdim,hiddendim], dtype=tf.float32)
bias1=tf.get_variable("b1", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight2=tf.get_variable("w2", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias2=tf.get_variable("b2", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight3=tf.get_variable("w3", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias3=tf.get_variable("b3", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight4=tf.get_variable("w4", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,outputdim], dtype=tf.float32)
bias4=tf.get_variable("b4", initializer=tf.initializers.random_normal(0,0.00), shape=[outputdim], dtype=tf.float32)

#varlist=[weight1,weight2,weight3,weight4,bias1,bias2,bias3,bias4]
varlist=[weight1,weight4,bias1,bias4]
H1=tf.nn.relu(tf.nn.xw_plus_b(X,weight1,bias1))
#H2=tf.nn.relu(tf.nn.xw_plus_b(H1,weight2,bias2))
#H3=tf.nn.relu(tf.nn.xw_plus_b(H2,weight3,bias3))
#Y_pred=tf.nn.xw_plus_b(H3,weight4,bias4)
Y_pred=tf.nn.xw_plus_b(H1,weight4,bias4)


# Y=tf.placeholder(tf.float32,shape=[None,1],name="Y")
# action=tf.placeholder(tf.int32,shape=[1],name="action")

weight01=tf.get_variable("w01", initializer=tf.keras.initializers.he_uniform(), shape=[inputdim,hiddendim], dtype=tf.float32)
bias01=tf.get_variable("b01", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight02=tf.get_variable("w02", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias02=tf.get_variable("b02", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight03=tf.get_variable("w03", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias03=tf.get_variable("b03", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight04=tf.get_variable("w04", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,outputdim], dtype=tf.float32)
bias04=tf.get_variable("b04", initializer=tf.initializers.random_normal(0,0.00), shape=[outputdim], dtype=tf.float32)

#varlist0=[weight01,weight02,weight03,weight04,bias01,bias02,bias03,bias04]
#varlist=[weight01,weight04,bias01,bias04]
H01=tf.nn.relu(tf.nn.xw_plus_b(X,weight01,bias01))
#H02=tf.nn.relu(tf.nn.xw_plus_b(H01,weight02,bias02))
#H03=tf.nn.relu(tf.nn.xw_plus_b(H02,weight03,bias03))
#Y_pred0=tf.nn.xw_plus_b(H03,weight04,bias04)
Y_pred0=tf.nn.xw_plus_b(H01,weight04,bias04)

assw1=weight01.assign(weight1)
assw2=weight02.assign(weight2)
assw3=weight03.assign(weight3)
assw4=weight04.assign(weight4)
assb1=bias01.assign(bias1)
assb2=bias02.assign(bias2)
assb3=bias03.assign(bias3)
assb4=bias04.assign(bias4)

#loss=tf.square(Y-Y_pred)
loss=tf.reduce_mean(tf.square(Y-Y_pred))
# loss=tf.square(Y-Y_pred[action])
#opt=tf.train.AdadeltaOptimizer(1.0).minimize(loss,var_list=varlist)
#opt=tf.train.GradientDescentOptimizer(lr).minimize(loss,var_list=varlist)
opt=tf.train.AdamOptimizer(lr).minimize(loss,var_list=varlist)
#opt=tf.train.AdamOptimizer(lr).minimize(loss)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)



16 4


In [24]:
max_episodes=200
max_step=100
gamma=0.995
epsilon0 = 0.5
sess.run(init)
sess.run([assw1,assw2,assw3,assw4,assb1,assb2,assb3,assb4])
# initialize

epsilon=epsilon0
min_step=max_step
min_step_episode=max_episodes
min_step_path=[]
max_replay=1000
replay=deque(maxlen=max_replay)
i_step=0
batch_size=100

for episode in range(max_episodes):
    epsilon=epsilon*0.99
    state = env.reset()
    total_reward = 0
    state_list = [state]
    win=0
    
    state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
    state_m[0][state]=1.0
    Q_pred=sess.run(Y_pred0,feed_dict={X:state_m})
    if np.random.rand()>epsilon :
        action=np.argmax(Q_pred[0])
    else:
        action=np.random.choice(env.action_space.n) 
    for step in range(1,max_step+1):
        
        i_step+=1
   
        state_next,reward,game_over,_ = env.step(action)
        
        if reward==0 : 
            reward=-0.01
        if reward!=1.0 and game_over==1:
            reward=-1
        
        state_m_next=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m_next[0][state_next]=1.0
        Q_pred_next=sess.run(Y_pred0, feed_dict={X:state_m_next}) 
        if np.random.rand()>epsilon :
            action_next=np.argmax(Q_pred_next[0])
        else:
            action_next=np.random.choice(env.action_space.n)
        
        Y_true=copy.deepcopy(Q_pred[0])
        if reward==1.0 and game_over==1:
            Y_true[action]= reward
        else :
            Y_true[action]=reward+gamma* (Q_pred_next[0][action_next])
        Y_true=np.reshape(Y_true,[1,env.action_space.n])
        replay.append([state_m[0],Y_true])
        

#            game_over=0
        # Q-Learning
        if i_step%100==0 and i_step!=0:
            mini_batch = random.sample(replay,batch_size)
            states=np.zeros([batch_size,inputdim])
            Qs=np.zeros([batch_size,outputdim])
            for m in range(batch_size):
                states[m]=mini_batch[m][0]
                Qs[m]=mini_batch[m][1]
            cost0= sess.run([loss],feed_dict={X:states,Y:Qs})
            for i in range(0,5):
                sess.run([opt],feed_dict={X:states,Y:Qs})
                cost= sess.run([loss],feed_dict={X:states,Y:Qs})
#            Q_pred=sess.run(Y_pred,feed_dict={X:states})
            #print(i_step,cost0,cost,Qs[0],Q_pred[0])
            print(i_step,cost0,cost)

        if i_step%300==0 :
            print(i_step,"update")
            sess.run([assw1,assw2,assw3,assw4,assb1,assb2,assb3,assb4])
            
        state=state_next
        state_m=state_m_next
        action=action_next
        
        total_reward=total_reward+reward
        state_list.append(state)
        if game_over and reward==1:
            win=1
            if step< min_step:
                min_step=step
                min_step_path=state_list
                min_step_episode=episode
        if game_over==1:
            break
            

    line_out="Episode: %d, step: %d, total_reward: %f, epsilon: %f" %(episode, step, total_reward,epsilon)
    print(line_out)
    if reward==1 :
        print(state_list)
print( "min step: ", min_step, "min step episode", min_step_episode)
print("shortest path: ", min_step_path)
#print(Q)

Episode: 0, step: 13, total_reward: -1.120000, epsilon: 0.495000
Episode: 1, step: 11, total_reward: -1.100000, epsilon: 0.490050
Episode: 2, step: 10, total_reward: -1.090000, epsilon: 0.485149
Episode: 3, step: 12, total_reward: -1.110000, epsilon: 0.480298
Episode: 4, step: 31, total_reward: -1.300000, epsilon: 0.475495
Episode: 5, step: 8, total_reward: -1.070000, epsilon: 0.470740
100 [0.12620485] [0.05367122]
Episode: 6, step: 23, total_reward: -1.220000, epsilon: 0.466033
Episode: 7, step: 24, total_reward: -1.230000, epsilon: 0.461372
Episode: 8, step: 22, total_reward: -1.210000, epsilon: 0.456759
Episode: 9, step: 11, total_reward: -1.100000, epsilon: 0.452191
Episode: 10, step: 4, total_reward: -1.030000, epsilon: 0.447669
Episode: 11, step: 21, total_reward: -1.200000, epsilon: 0.443192
200 [0.070781566] [0.06756154]
Episode: 12, step: 14, total_reward: -1.130000, epsilon: 0.438761
Episode: 13, step: 8, total_reward: 0.930000, epsilon: 0.434373
[0, 0, 0, 1, 2, 6, 10, 14, 15

Episode: 110, step: 29, total_reward: -1.280000, epsilon: 0.163861
Episode: 111, step: 7, total_reward: -1.060000, epsilon: 0.162223
2500 [0.015939679] [0.013319025]
Episode: 112, step: 72, total_reward: -1.710000, epsilon: 0.160601
Episode: 113, step: 54, total_reward: -1.530000, epsilon: 0.158995
Episode: 114, step: 27, total_reward: -1.260000, epsilon: 0.157405
2600 [0.01586553] [0.013343914]
Episode: 115, step: 24, total_reward: -1.230000, epsilon: 0.155831
Episode: 116, step: 34, total_reward: -1.330000, epsilon: 0.154272
Episode: 117, step: 30, total_reward: -1.290000, epsilon: 0.152730
2700 [0.012928121] [0.012597285]
2700 update
Episode: 118, step: 16, total_reward: -1.150000, epsilon: 0.151202
Episode: 119, step: 17, total_reward: -1.160000, epsilon: 0.149690
Episode: 120, step: 57, total_reward: -1.560000, epsilon: 0.148193
Episode: 121, step: 2, total_reward: -1.010000, epsilon: 0.146711
2800 [0.023890013] [0.022146253]
Episode: 122, step: 38, total_reward: -1.370000, epsilo

In [25]:
#state=np.arange(0,15)
#sess.run(init)
state_m=np.zeros([16,env.observation_space.n],dtype=np.float32)
for i in range(0,15):
    state_m[i][i]=1.0
print(state_m)
Q_pred=sess.run(Y_pred,feed_dict={X:state_m})
print(Q_pred)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-0.3193047   0.0772663   0.12334345  0.1268539 ]
 [-0.32311273  0.09331876  0.17124411  0.11340132]
 [-0.3256859   0.06620821  0.1321699   0.12647435]
 [-0.32298675  0.05508754  0.1